# クイックスタート: Mosaic AI Agent Frameworkを使用してエージェントを構築、テスト、デプロイする
このクイックスタートノートブックでは、Mosaic AI Agent Framework ([AWS](https://docs.databricks.com/aws/ja/generative-ai/guide/introduction-generative-ai-apps#what-are-gen-ai-apps) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/guide/introduction-generative-ai-apps#what-are-gen-ai-apps) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/guide/introduction-generative-ai-apps)) を使用して、Databricks上で生成AIエージェントを構築、テスト、デプロイする方法を示します。([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/build-genai-apps#-mosaic-ai-agent-framework))

## エージェントの定義とテスト
このセクションでは、以下の属性を持つシンプルなエージェントを定義し、テストします。

- エージェントは、Databricks Foundation Model APIで提供されるLLMを使用します。([AWS](https://docs.databricks.com/aws/ja/machine-learning/foundation-model-apis) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/machine-learning/foundation-model-apis/) | [GCP](https://docs.databricks.com/gcp/ja/machine-learning/foundation-model-apis))
- エージェントは、Databricks Unity Catalog上の組み込みPythonコードインタープリターツールにアクセスできます。このツールを使用して、ユーザーの質問に応答するためにLLM生成コードを実行できます。([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/code-interpreter-tools#built-in-python-executor-tool) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/code-interpreter-tools) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/code-interpreter-tools))

LLMエンドポイントをクエリするために`databricks_openai` SDKを使用します。([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/author-agent#requirements) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/author-agent#requirements) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/author-agent#requirements))

In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()

In [0]:
# 以下のスニペットは、候補の中からDatabricksワークスペースで利用可能な最初のLLM APIを選択しようとします。
# LLM_ENDPOINT_NAMEを指定するだけにオーバーライドして簡略化できます。
LLM_ENDPOINT_NAME = None

from databricks.sdk import WorkspaceClient
def is_endpoint_available(endpoint_name):
  try:
    client = WorkspaceClient().serving_endpoints.get_open_ai_client()
    client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "AIとは何ですか？"}])
    return True
  except Exception:
    return False
  
client = WorkspaceClient()
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
      LLM_ENDPOINT_NAME = candidate_endpoint_name
assert LLM_ENDPOINT_NAME is not None, "LLM_ENDPOINT_NAMEを指定してください"

In [0]:
import json
import mlflow
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient

# LLM呼び出しからのトレースを自動的にログに記録してデバッグを容易にする
mlflow.openai.autolog()

# Databricksモデルサービングエンドポイントと通信するように構成されたOpenAIクライアントを取得
# これを使用してエージェント内のLLMにクエリを送信する
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# Databricks組み込みツールをロード（ステートレスなPythonコードインタープリターツール）
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(
    function_names=["system.ai.python_exec"], client=client
).tools
for tool in builtin_tools:
    del tool["function"]["strict"]


def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function(
            "system.ai.python_exec", parameters=parameters
        )
    raise ValueError(f"未知のツール: {tool_name}")


def run_agent(prompt):
    """
    ユーザープロンプトをLLMに送信し、LLMの応答メッセージのリストを返す
    LLMは必要に応じてコードインタープリターツールを呼び出してユーザーに応答することができる
    """
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # モデルがツールを実行した場合、それを呼び出す
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
        result_msgs.append(
            {
                "role": "tool",
                "content": tool_result.value,
                "name": call.function.name,
                "tool_call_id": call.id,
            }
        )
    return result_msgs

In [0]:
answer = run_agent("429の平方根は何ですか？")
for message in answer:
    print(f'{message["role"]}: {message["content"]}')

## エージェントコードの準備

エージェントの定義をMLflowの[ChatAgentインターフェース](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent)でラップして、コードをログに記録する準備をします。

MLflowの標準エージェント作成インターフェースを使用することで、エージェントとチャットしたり、デプロイ後に他の人と共有したりするための組み込みUIを利用できます。([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/author-agent#-use-chatagent-to-author-agents) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/author-agent) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/author-agent))

In [0]:
import uuid
import mlflow
from typing import Any, Optional

from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

mlflow.openai.autolog()

class QuickstartAgent(ChatAgent):
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # 1. 入力メッセージから最後のユーザープロンプトを抽出
        prompt = messages[-1].content

        # 2. run_agentを呼び出して、応答メッセージのリストを取得
        raw_msgs = run_agent(prompt)

        # 3. 各応答メッセージをChatAgentMessageにマップし、応答を返す
        out = []
        for m in raw_msgs:
            out.append(ChatAgentMessage(id=uuid.uuid4().hex, **m))

        return ChatAgentResponse(messages=out)

In [0]:
AGENT = QuickstartAgent()
for response_message in AGENT.predict(
    {"messages": [{"role": "user", "content": "429の平方根は何ですか？"}]}
).messages:
    print(f"role: {response_message.role}, content: {response_message.content}")

## エージェントの記録

エージェントをログに記録し、Unity Catalogにモデルとして登録します ([AWS](https://docs.databricks.com/aws/ja/machine-learning/manage-model-lifecycle/) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/machine-learning/manage-model-lifecycle/) | [GCP](https://docs.databricks.com/gcp/ja/machine-learning/manage-model-lifecycle/))。このステップでは、エージェントコードとその依存関係を単一のアーティファクトにパッケージ化し、サービングエンドポイントにデプロイします。

以下のコードセルは次のことを行います：

1. 上記のエージェントコードをコピーして、単一のセルに結合します。
1. セルの先頭に `%%writefile` セルマジックコマンドを追加して、エージェントコードを `quickstart_agent.py` というファイルに保存します。
1. セルの下部に [mlflow.models.set_model()](https://mlflow.org/docs/latest/model#models-from-code) 呼び出しを追加します。これにより、エージェントがデプロイされたときに予測を行うために使用するPythonエージェントオブジェクトをMLflowに伝えます。
1. MLflow APIを使用して `quickstart_agent.py` ファイル内のエージェントコードをログに記録します ([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/log-agent) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/log-agent) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/log-agent))。

In [0]:
%%writefile quickstart_agent.py

import json
import uuid
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, DatabricksFunctionClient
from typing import Any, Optional

import mlflow
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentMessage, ChatAgentResponse, ChatContext

# Databricksのモデルサービングエンドポイントと通信するために設定されたOpenAIクライアントを取得
# これを使ってエージェント内でLLMに問い合わせる
openai_client = WorkspaceClient().serving_endpoints.get_open_ai_client()

# 以下のスニペットは、Databricksワークスペース内で利用可能な最初のLLM APIを
# 複数の候補から選択しようとします。必要に応じてLLM_ENDPOINT_NAMEを直接指定して簡略化可能です。
LLM_ENDPOINT_NAME = None

def is_endpoint_available(endpoint_name):
  try:
    client = WorkspaceClient().serving_endpoints.get_open_ai_client()
    client.chat.completions.create(model=endpoint_name, messages=[{"role": "user", "content": "AIとは何ですか？"}])
    return True
  except Exception:
    return False
  
for candidate_endpoint_name in ["databricks-claude-3-7-sonnet", "databricks-meta-llama-3-3-70b-instruct"]:
    if is_endpoint_available(candidate_endpoint_name):
      LLM_ENDPOINT_NAME = candidate_endpoint_name
assert LLM_ENDPOINT_NAME is not None, "LLM_ENDPOINT_NAMEを指定してください"

# LLM呼び出しの自動トレースを有効化
mlflow.openai.autolog()

# Databricks組み込みツール（ステートレスなPythonコードインタプリターツール）をロード
client = DatabricksFunctionClient()
builtin_tools = UCFunctionToolkit(function_names=["system.ai.python_exec"], client=client).tools
for tool in builtin_tools:
    del tool["function"]["strict"]

def call_tool(tool_name, parameters):
    if tool_name == "system__ai__python_exec":
        return DatabricksFunctionClient().execute_function("system.ai.python_exec", parameters=parameters)
    raise ValueError(f"不明なツールです: {tool_name}")

def run_agent(prompt):
    """
    ユーザープロンプトをLLMに送信し、LLMの応答メッセージのリストを返す
    必要に応じて、LLMはコードインタプリターツールを呼び出してユーザーに応答可能
    """
    result_msgs = []
    response = openai_client.chat.completions.create(
        model=LLM_ENDPOINT_NAME,
        messages=[{"role": "user", "content": prompt}],
        tools=builtin_tools,
    )
    msg = response.choices[0].message
    result_msgs.append(msg.to_dict())

    # モデルがツールを実行した場合は呼び出す
    if msg.tool_calls:
        call = msg.tool_calls[0]
        tool_result = call_tool(call.function.name, json.loads(call.function.arguments))
        result_msgs.append({"role": "tool", "content": tool_result.value, "name": call.function.name, "tool_call_id": call.id})
    return result_msgs

class QuickstartAgent(ChatAgent):
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        prompt = messages[-1].content
        raw_msgs = run_agent(prompt)
        out = []
        for m in raw_msgs:
            out.append(ChatAgentMessage(
                id=uuid.uuid4().hex,
                **m
            ))

        return ChatAgentResponse(messages=out)

AGENT = QuickstartAgent()
mlflow.models.set_model(AGENT)

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution
from quickstart_agent import LLM_ENDPOINT_NAME

# モデルをワークスペースのデフォルトカタログに登録
# 必要に応じてカタログ（例: "main"）およびスキーマ名（例: "custom_schema"）を指定し、
# エージェントを別の場所に登録する
catalog_name = "takaakiyayoi_catalog"
schema_name = "databricks_101"
registered_model_name = f"{catalog_name}.{schema_name}.quickstart_agent"

# Databricks製品リソースを指定し、エージェントがこれらのリソースにアクセスできるようにする
# （組み込みのPythonコードインタプリターツールとLLMサービングエンドポイント）
# これにより、エージェントがデプロイされる際にDatabricksが自動的に認証を構成できるようにする
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksFunction(function_name="system.ai.python_exec"),
]

mlflow.set_registry_uri("databricks-uc")
with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="quickstart_agent.py",
        extra_pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}"
        ],
        resources=resources,
        registered_model_name=registered_model_name,
    )

## エージェントのデプロイ

以下のセルを実行してエージェントをデプロイします ([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-framework/deploy-agent) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-framework/deploy-agent) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-framework/deploy-agent))。エージェントエンドポイントが起動すると、AI Playground ([AWS](https://docs.databricks.com/aws/ja/large-language-models/ai-playground) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/large-language-models/ai-playground) | [GCP](https://docs.databricks.com/gcp/ja/large-language-models/ai-playground)) を介してエージェントとチャットしたり、ステークホルダーと共有して初期フィードバックを得たり ([AWS](https://docs.databricks.com/aws/ja/generative-ai/agent-evaluation/review-app) | [Azure](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/agent-evaluation/review-app) | [GCP](https://docs.databricks.com/gcp/ja/generative-ai/agent-evaluation/review-app)) することができます。

In [0]:
from databricks import agents

deployment_info = agents.deploy(
    model_name=registered_model_name,
    model_version=logged_agent_info.registered_model_version,
)